In [1]:
%pip install bertopic

  Using cached PyYAML-5.4.1-cp310-cp310-win_amd64.whl
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
Note: you may need to restart the kernel to use updated packages.


'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pandas as pd
print()

import spacy
print()

import en_core_web_sm
nlp = en_core_web_sm.load()

In [3]:
def get_names(df_plot_f):
    list_names = []
    for i, row in df_plot_f.iterrows():
        doc_nlp = nlp(row['plot'])
        text_label = [(X.text, X.label_) for X in doc_nlp.ents]
        people = [i for i,label in text_label if 'PERSON' in label]
        people = list(set(people))

        list_names.extend(people)
        
    return list_names

In [22]:
df_plot = pd.read_csv('df_plot.csv')
df_clean = pd.read_csv('df_clean.csv')

In [77]:
df_plot = df_plot[~df_plot['plot'].isin(["http"])]

In [78]:
docs = df_plot.sample(frac=1).reset_index()['plot'][:300]

In [79]:
#get names from plots to add them to stopwords
list_names = get_names(docs)

AttributeError: 'Series' object has no attribute 'iterrows'

In [ ]:
#!pip install pyyaml
import bertopic

In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

In [ ]:
umap_model = UMAP()
hdbscan_model = HDBSCAN(prediction_data=True, gen_min_span_tree=True)

#tunable yperparams 
#umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
#hdbscan_model = HDBSCAN(min_cluster_size=15,min_samples=5,
#                        prediction_data=True, gen_min_span_tree=True)

In [ ]:
from sentence_transformers import SentenceTransformer

# use this to apply embedding on plots - time consuming! 
embedding_model = "all-MiniLM-L6-v2"
# embedding_model = "all-mpnet-base-v2"  #big model, better precision, bigger runtime

In [76]:
from nltk.corpus import stopwords

stopwords_custom = ['http', 'https', 'amp', 'com','one','film', 'two', 'man', 'films']
# stopwords without names  
stopwords = list(stopwords.words('english')) + stopwords_custom #+ list_names
# stopwords with names
#stopwords = list(stopwords.words('english')) + ['http', 'https', 'amp', 'com'] + list_names
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

# Create topic model
model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    top_n_words=15,
    language='english',
    calculate_probabilities=True,
    verbose=False,
)
topics, probs = model.fit_transform(docs)

2022-12-19 19:43:05,155 - BERTopic - Transformed documents to Embeddings
2022-12-19 19:43:07,785 - BERTopic - Reduced dimensionality
2022-12-19 19:43:07,826 - BERTopic - Clustered reduced embeddings


In [ ]:
print('there are', probs.shape[1], 'topics, for', probs.shape[0], 'films.')

there are  2 topics, for  300 films.


In [80]:
model.visualize_documents(docs)

In [71]:
model.visualize_topics(model.topics_)

In [72]:
model.visualize_barchart()

In [73]:
model.get_topic_info()

,Topic,Count,Name
0,-1,2,-1_bond_drax_jaws_goodhead
1,0,291,0_two_man_back_life
2,1,7,1_islam_waters_experiences_films


In [74]:
model.visualize_hierarchy()

In [ ]:
#umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
#hdbscan_model = HDBSCAN(min_cluster_size=15,min_samples=5,
#                        prediction_data=True, gen_min_span_tree=True)

umap_model = UMAP()
hdbscan_model = HDBSCAN(prediction_data=True, gen_min_span_tree=True)

sert a rien en dessous

In [ ]:
model.reduce_topics(df_shuffle_test['plot'], nr_topics=40)

In [ ]:
model.visualize_documents(docs=df_shuffle_test['plot'],embeddings=embeddings)

In [ ]:
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=20,min_samples=5,  gen_min_span_tree=True)
clusterer.fit(embeddings)

clusterer.condensed_tree_.plot(select_clusters=True)


In [ ]:
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=7,min_samples=1, gen_min_span_tree=True)
clusterer.fit(embeddings)

clusterer.condensed_tree_.plot(select_clusters=True)